### Import dependencies

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader

from datasets import ASLTestDataset
from model import ASLModel

### Define the device to use to in testing

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

### Initialize the model

In [3]:
model = ASLModel().to(device=device)
model.eval()

ASLModel(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=(3, 3), stride=(3, 3), padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(64, 72, kernel_size=(3, 3), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=(3, 3), stride=(3, 3), padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(72, 80, kernel_size=(3, 3), stride=(1, 1))
  (pool3): MaxPool2d(kernel_size=(3, 3), stride=(3, 3), padding=0, dilation=1, ceil_mode=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=2880, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=29, bias=True)
)

Load the previous trained model

In [4]:
model.load_state_dict(torch.load("./models/dev.pt"))

<All keys matched successfully>

### Load Test Dataset

In [5]:
test_dataset = ASLTestDataset()
test_dataloader = DataLoader(dataset=test_dataset, batch_size=128)

###  Define the Loss and Optimizer

In [6]:
from torch.optim import Adam
from torch.nn import CrossEntropyLoss

loss = CrossEntropyLoss()
optimizer = Adam(model.parameters())

### Model testing

Define testing function

In [7]:
from torcheval.metrics.functional import multiclass_f1_score
from torch.nn import CrossEntropyLoss

In [8]:
loss = CrossEntropyLoss()

In [13]:
from sklearn.metrics import f1_score

In [18]:
def test(dataloader: DataLoader, model: nn.Module):
    for i, (x, y) in enumerate(dataloader):
        with torch.no_grad():
            x =  x.to(device)
            y =  y.to(device)
            print(f"x.shape {x.shape}", f"y.shape: {y.shape}")
            yhat = model(x)
            criterion = loss(yhat, y)
            score = multiclass_f1_score(yhat, y, num_classes=29)
            # f1score = f1_score(yhat.cpu(), y.cpu())
            print(f"loss: {criterion.item()} | f1-score-1: {score} | f1-score-2:")


In [19]:
test(dataloader=test_dataloader, model=model)

x.shape torch.Size([28, 3, 200, 200]) y.shape: torch.Size([28])
loss: 9.665379524230957 | f1-score-1: 0.3214285671710968 | f1-score-2:
